In [ ]:
from IPython.display import HTML, Javascript, clear_output
import os
import sys

from google.colab import drive

drive.mount("/content/drive")

!pip install human_id
!pip install mlflow

# Define the repo URL and folder name
REPO_URL = "https://github.com/rosinusserrano/autoencoding_experiments.git"
LOCAL_DIR = "/content/autoencoding_experiments"

# Inject a floating button with JavaScript
def inject_floating_button():
    display(HTML("""
    <script>
    function reloadRepo() {
        google.colab.kernel.invokeFunction('notebook.reload_repo', [], {});
    }
    const button = document.createElement('button');
    button.innerHTML = 'Reload Repo';
    button.style.position = 'fixed';
    button.style.bottom = '20px';
    button.style.right = '20px';
    button.style.backgroundColor = '#ff5050';
    button.style.color = 'white';
    button.style.border = 'none';
    button.style.padding = '10px 20px';
    button.style.borderRadius = '5px';
    button.style.boxShadow = '0px 4px 6px rgba(0,0,0,0.1)';
    button.style.cursor = 'pointer';
    button.onclick = reloadRepo;
    document.body.appendChild(button);
    </script>
    """))

def clone_and_import():
    # Clear output and display new button
    clear_output(wait=True)
    inject_floating_button()
    # Clone the repository if it doesn't exist
    if not os.path.exists(LOCAL_DIR):
        !git clone {REPO_URL} {LOCAL_DIR}
    else:
        # Pull the latest changes if the repo already exists
        !cd {LOCAL_DIR} && git pull

    # Append the repository to the system path
    if LOCAL_DIR not in sys.path:
        sys.path.append(LOCAL_DIR + "/src")
    print(f"Repository at '{LOCAL_DIR}' is ready for imports.")

# Register the Python function as a callable function for JavaScript
from google.colab import output
output.register_callback('notebook.reload_repo', clone_and_import)

# Inject the button
inject_floating_button()

# Initial setup
clone_and_import()

In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
from functools import partial

from datasets import DatasetConfig
from logger.caveman import CavemanLogger
from logger.mlflow import MlFlowLogger
from models.vae import VAEConfig, mse_and_kld_loss
from models.autoencoder import AutoencoderV2Config, mse_loss
from utils.train import TrainConfig, standard_training_pipeline

import mlflow

In [11]:
model_config = AutoencoderV2Config()

dataset_config = DatasetConfig(
    root="/content/drive/MyDrive/Autoencoding Experiments/datasets/STL10",
    dataset_name="stl10",
    validation_split=0.1,
    batch_size=64,
)
train_config = TrainConfig("adam", learning_rate=0.0005, n_epochs=3, weight_decay=0.0001,)

experiment_name = "AutoencoderV2 on STL10"

# logger = CavemanLogger(f"../runs/{experiment_name}")
# logger = CavemanLogger(f"/content/drive/MyDrive/Autoencoding Experiments/{experiment_name}")

mlflow.end_run()
logger = MlFlowLogger(experiment_name=experiment_name, remote_url="https://mlflow.sniggles.de", debug=True)

standard_training_pipeline(
    model_config=model_config,
    dataset_config=dataset_config,
    train_config=train_config,
    logger=logger,
    loss_fn=mse_loss,
    validation_interval=1,
    test_interval=1,
    visualization_interval=1,
)

🏃 View run tasteful-hare-79 at: https://mlflow.sniggles.de/#/experiments/332547331281273096/runs/b927e60092754d609a41f0a02042cd95
🧪 View experiment at: https://mlflow.sniggles.de/#/experiments/332547331281273096
Setting tracking uri for mlflow
Created experiment with id 332547331281273096
Starting run
Files already downloaded and verified
Files already downloaded and verified
Logging configs.


  0%|          | 0/1407 [00:00<?, ?it/s]

TypeError: 'NoneType' object is not callable